## 0. Imports and code config

In [43]:
# === 0) Imports & core config ===
# !pip install -q pybioclip scikit-learn pandas numpy pillow torch

import json, random, hashlib, os
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from PIL import Image
import torch

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# BioCLIP: used as a feature extractor (image -> embedding vector)
from bioclip.predict import BaseClassifier

# ---- paths (adjust these if needed) ----
CLASS_NAMES = "../data/processed/class_names_no_ixodes.json"   # species list
DATA   = "../data/processed/final_data_no_ixodes.json"    # manifest with image_path, true_label, sample_id, view

# ---- few-shot split params ----
K         = 5     # train specimens per species
MIN_TOTAL = 6     # species must have >=6 specimens (so >=1 test)
SEED      = 0     # seed for single dry run; we’ll add 0..99 later

# ---- per-image embedding cache (under data/processed/) ----
EMB_CACHE = Path("..") / "data" / "processed" / "emb_cache"
EMB_CACHE.mkdir(parents=True, exist_ok=True)  # create if missing
print("Embedding cache dir:", EMB_CACHE.resolve())

# ---- per-seed results root (under data/processed/) ----
RESULTS_ROOT = Path("..") / "results" / "svm_bioclip"
RESULTS_ROOT.mkdir(parents=True, exist_ok=True)
print("Results root dir:", RESULTS_ROOT.resolve())

# ---- device + model init ----
DEVICE = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
BC = BaseClassifier(device=DEVICE)

print("Device:", DEVICE)

Embedding cache dir: /Users/jayjivandas/Research_Imageomics/BioClip/tick_classification_project/data/processed/emb_cache
Results root dir: /Users/jayjivandas/Research_Imageomics/BioClip/tick_classification_project/results/svm_bioclip
Device: mps


## 1. Load classes and Specimens

In [44]:
# === Block 1 — Load Classes and Data → Build Specimen Index ===

# 1) Load class list and data
with open(CLASS_NAMES, "r") as f:   # CLASS_NAMES was set in Block 0 to the JSON path
    SPECIES_LIST = set(json.load(f))  # species names we care about

with open(DATA, "r") as f:          # DATA was set in Block 0 to the JSON path
    data = json.load(f)             # list of records (image_path, true_label, sample_id, view)

# 2) Build specimen index:
#    by_species[species][sample_id] -> {"dorsal": <path>, "ventral": <path>}
by_species = defaultdict(lambda: defaultdict(dict))
for r in data:
    sp = r["true_label"]
    if sp not in SPECIES_LIST:
        continue
    sid  = r["sample_id"]
    view = str(r["view"]).strip().lower()
    by_species[sp][sid][view] = r["image_path"]

# 3) Keep only specimens with BOTH views
for sp in list(by_species.keys()):
    for sid in list(by_species[sp].keys()):
        views = by_species[sp][sid]
        if not ("dorsal" in views and "ventral" in views):
            del by_species[sp][sid]
    if not by_species[sp]:
        del by_species[sp]

# 4) Keep only species with >= MIN_TOTAL specimens
for sp in list(by_species.keys()):
    if len(by_species[sp]) < MIN_TOTAL:
        del by_species[sp]

included_species = sorted(by_species.keys())

# 5) Quick summary
n_specimens = sum(len(smap) for smap in by_species.values())
print(f"Included species: {len(included_species)}")
print(f"Total usable specimens (both views, species >= {MIN_TOTAL}): {n_specimens}")

# Optional: per-species counts
species_counts = {sp: len(smap) for sp, smap in by_species.items()}
for sp in included_species:
    print(f"{sp}: {species_counts[sp]} specimens")

assert len(included_species) > 0, "No species meet the criteria (both views and MIN_TOTAL)."

Included species: 5
Total usable specimens (both views, species >= 6): 578
Amblyomma americanum: 90 specimens
Amblyomma maculatum: 6 specimens
Dermacentor variabilis: 308 specimens
Haemaphysalis longicornis: 35 specimens
Ixodes scapularis: 139 specimens


## Block 2 Train test split

In [45]:
# === Block 2 — Train/Test Split (5-shot, Seeded, no leakage) ===

def split_once(by_species, species_list, k=K, seed=SEED):
    rng = random.Random(seed)
    train_pairs, test_pairs = [], []
    per_species_counts = {}

    for sp in species_list:
        sids = list(by_species[sp].keys())
        rng.shuffle(sids)
        tr, te = sids[:k], sids[k:]

        # Sanity checks per your policy
        assert len(tr) == k, f"{sp}: needs exactly {k} train specimens, found {len(tr)}"
        assert len(te) >= 1, f"{sp}: needs at least 1 test specimen (has {len(sids)} total)"

        train_pairs.extend([(sp, sid) for sid in tr])
        test_pairs.extend([(sp, sid) for sid in te])
        per_species_counts[sp] = {"train": len(tr), "test": len(te), "total": len(sids)}

    # No leakage: specimen (sample_id) cannot be in both sets
    assert not (set(train_pairs) & set(test_pairs)), "Leakage detected: same (species, sample_id) in train & test"

    return train_pairs, test_pairs, per_species_counts

train_pairs, test_pairs, split_counts = split_once(by_species, included_species, k=K, seed=SEED)

# ---- Summary prints ----
n_species = len(included_species)
n_train   = len(train_pairs)
n_test    = len(test_pairs)

print(f"Seed: {SEED}")
print(f"Species included: {n_species}")
print(f"Train specimens: {n_train}  (expected {K} × {n_species} = {K*n_species})")
print(f"Test specimens:  {n_test}")

for sp in included_species:
    c = split_counts[sp]
    print(f"  {sp}: total={c['total']}, train={c['train']}, test={c['test']}")

# Optional: peek a few chosen IDs per species (trim for readability)
for sp in included_species:
    chosen = [sid for (s, sid) in train_pairs if s == sp][:min(3, K)]
    print(f"  Train IDs sample — {sp}: {chosen}")

Seed: 0
Species included: 5
Train specimens: 25  (expected 5 × 5 = 25)
Test specimens:  553
  Amblyomma americanum: total=90, train=5, test=85
  Amblyomma maculatum: total=6, train=5, test=1
  Dermacentor variabilis: total=308, train=5, test=303
  Haemaphysalis longicornis: total=35, train=5, test=30
  Ixodes scapularis: total=139, train=5, test=134
  Train IDs sample — Amblyomma americanum: ['ZOE-0014-12', 'ZOE-0014-11', '201-01']
  Train IDs sample — Amblyomma maculatum: ['GLN-0087-06', 'GLN-0087-10', 'GLN-0087-07']
  Train IDs sample — Dermacentor variabilis: ['435-01', '359-01', '147-01']
  Train IDs sample — Haemaphysalis longicornis: ['47-02', 'GLN-0087-15', 'OPL-0098-01']
  Train IDs sample — Ixodes scapularis: ['55-10', '60-01', '221-01']


## Block 3: Embedding helpers, getting the embeddings and building our Cache. 

In [46]:
# === Block 3 — Embedding Helpers (Per-Image Cache + Specimen Vector) ===

# Per-image cache filename (hash of the image path -> unique + reproducible)
def _cache_fp(img_path: str) -> Path:
    h = hashlib.sha256(img_path.encode("utf-8")).hexdigest()[:24]
    return EMB_CACHE / f"{h}.npy"

# Single-image -> embedding (uses cache if available)
def embed_image(img_path: str) -> np.ndarray:
    """
    Input:  path to an image file
    Output: 1D numpy array (BioCLIP embedding), L2-normalized if normalize=True in create_image_features
    """
    fp = _cache_fp(img_path)
    if fp.exists():
        return np.load(fp)
    pil = Image.open(img_path).convert("RGB")
    vec = BC.create_image_features([pil], normalize=True).cpu().numpy()[0]
    np.save(fp, vec)
    return vec

# Specimen-level vector: average dorsal + ventral embeddings -> one vector per specimen
def specimen_vec(rec: dict) -> np.ndarray:
    """
    rec is: {"dorsal": <path>, "ventral": <path>}
    Returns a single vector for the specimen: 0.5*(z_dorsal + z_ventral)
    """
    z_d = embed_image(rec["dorsal"])
    z_v = embed_image(rec["ventral"])
    return 0.5 * (z_d + z_v)

## Block 4- build X/Y, Train SVM, Metrics, SINGLE SEED

In [47]:
# === Block 4 — Build X/y, Train SVM, Predict, Quick Metrics (single seed) ===

# helper: turn (species, sample_id) pairs into X (embeddings) and y (labels)
def build_xy(by_species, pairs):
    X, y, ids = [], [], []
    for sp, sid in pairs:
        rec = by_species[sp][sid]          # {"dorsal":..., "ventral":...}
        X.append(specimen_vec(rec))        # averaged dorsal+ventral embedding
        y.append(sp)                       # species label
        ids.append(sid)                    # specimen ID for reporting
    return np.stack(X), np.array(y), ids

# assemble train/test matrices
Xtr, ytr, _      = build_xy(by_species, train_pairs)
Xte, yte, te_ids = build_xy(by_species, test_pairs)

# SVM pipeline (scale → RBF SVM); probability=True so we can get a confidence %
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf", class_weight="balanced", probability=True))
])

# train + predict
clf.fit(Xtr, ytr)
yhat  = clf.predict(Xte)
probs = clf.predict_proba(Xte)
conf  = probs.max(axis=1)   # top-class probability per specimen

# quick metrics
overall_acc = accuracy_score(yte, yhat)

# per-species accuracy → macro/balanced accuracy
tmp = pd.DataFrame({"true": yte, "pred": yhat})
per_species_acc = tmp.assign(hit=(tmp.true == tmp.pred).astype(int)).groupby("true")["hit"].mean()
macro_acc = per_species_acc.mean()

print(f"Overall accuracy: {overall_acc:.3f}")
print(f"Macro (balanced) accuracy: {macro_acc:.3f}")

# quick peek at a few specimen-level predictions
preview = pd.DataFrame({
    "sample_id": te_ids,
    "true_label": yte,
    "pred_label": yhat,
    "pred_confidence": conf
})
display(preview.head(10))

Overall accuracy: 0.814
Macro (balanced) accuracy: 0.814


,sample_id,true_label,pred_label,pred_confidence
0,ZOE-0014-15,Amblyomma americanum,Amblyomma americanum,0.436862
1,150-01,Amblyomma americanum,Amblyomma americanum,0.453386
2,69-02,Amblyomma americanum,Amblyomma americanum,0.372667
3,31-02,Amblyomma americanum,Amblyomma americanum,0.360671
4,43-15,Amblyomma americanum,Amblyomma americanum,0.411300
5,34-01,Amblyomma americanum,Amblyomma americanum,0.478879
6,61-01,Amblyomma americanum,Amblyomma americanum,0.387920
7,40-01,Amblyomma americanum,Amblyomma americanum,0.416312
8,465-01,Amblyomma americanum,Amblyomma americanum,0.402891
9,ZOE-0014-01,Amblyomma americanum,Amblyomma americanum,0.451165


## Block 5 Reporting and saving results for a single seed

In [48]:
# === Block 5 — Reporting & Save Artifacts (single seed) ===

# 1) Per-specimen results table (TEST set)
results = pd.DataFrame({
    "sample_id": te_ids,
    "true_label": yte,
    "pred_label": yhat,
})
if 'conf' in locals():
    results["pred_confidence"] = conf
results["correct"] = (results.true_label == results.pred_label).astype(int)

# 2) Per-species accuracy table
per_species = (results.assign(one=1)
               .groupby("true_label")
               .agg(n_test=("one", "sum"),
                    n_correct=("correct", "sum"))
               .assign(accuracy_species=lambda d: d.n_correct / d.n_test)
               .reset_index()
               .rename(columns={"true_label": "species"}))

# 3) Summary metrics
overall_acc = float(results["correct"].mean())
macro_acc   = float(per_species["accuracy_species"].mean())

# 4) Nicely display per-species accuracies in-notebook
per_species_display = (per_species
                       .sort_values(["accuracy_species", "n_test"], ascending=[False, False])
                       .reset_index(drop=True))
with pd.option_context('display.max_rows', None, 'display.float_format', '{:.3f}'.format):
    print(f"\nSeed {SEED} — Overall accuracy: {overall_acc:.3f} | Macro (balanced): {macro_acc:.3f}")
    print("Per-species accuracy (sorted):")
    display(per_species_display)

# 5) Prepare output folder for this seed (creates if missing)
#    If you moved RESULTS_ROOT under ../data/processed/, keep that same path from Block 0.
RESULTS_ROOT.mkdir(parents=True, exist_ok=True)
seed_dir = RESULTS_ROOT / f"seed_{SEED:03d}"
seed_dir.mkdir(parents=True, exist_ok=True)

# 6) Save CSVs
results.to_csv(seed_dir / "predictions.csv", index=False)
per_species.to_csv(seed_dir / "per_species.csv", index=False)

# 7) Save JSON summary + split for reproducibility
try:
    import bioclip as _bioclip_mod
    bioclip_version = getattr(_bioclip_mod, "__version__", "unknown")
except Exception:
    bioclip_version = "unknown"

summary = {
    "overall_accuracy": overall_acc,
    "macro_accuracy": macro_acc,
    "n_test": int(len(results)),
    "K": int(K),
    "MIN_TOTAL": int(MIN_TOTAL),
    "seed": int(SEED),
    "device": str(DEVICE),
    "bioclip_version": bioclip_version,
}
with open(seed_dir / "summary.json", "w") as f:
    json.dump(summary, f, indent=2)

with open(seed_dir / "split.json", "w") as f:
    json.dump({
        "train_pairs": train_pairs,   # list of [species, sample_id]
        "test_pairs":  test_pairs
    }, f, indent=2)

print("\nSaved files:")
print(" ", (seed_dir / "predictions.csv").resolve())
print(" ", (seed_dir / "per_species.csv").resolve())
print(" ", (seed_dir / "summary.json").resolve())
print(" ", (seed_dir / "split.json").resolve())


Seed 0 — Overall accuracy: 0.814 | Macro (balanced): 0.814
Per-species accuracy (sorted):


,species,n_test,n_correct,accuracy_species
0,Amblyomma maculatum,1,1,1.000
1,Amblyomma americanum,85,76,0.894
2,Ixodes scapularis,134,119,0.888
3,Dermacentor variabilis,303,239,0.789
4,Haemaphysalis longicornis,30,15,0.500



Saved files:
  /Users/jayjivandas/Research_Imageomics/BioClip/tick_classification_project/results/svm_bioclip/seed_000/predictions.csv
  /Users/jayjivandas/Research_Imageomics/BioClip/tick_classification_project/results/svm_bioclip/seed_000/per_species.csv
  /Users/jayjivandas/Research_Imageomics/BioClip/tick_classification_project/results/svm_bioclip/seed_000/summary.json
  /Users/jayjivandas/Research_Imageomics/BioClip/tick_classification_project/results/svm_bioclip/seed_000/split.json


## Block 6 Monte Carlo, 100 seeds. 

In [49]:
# === Block 6 — Monte Carlo (100 seeds), aggregate results ===

from tqdm import tqdm
import time

def run_one_seed(seed: int):
    """One complete SVM run for a given seed. Saves artifacts, returns metrics dict."""
    
    # 1. Split
    train_pairs, test_pairs, split_counts = split_once(by_species, included_species, k=K, seed=seed)

    # 2. Build embeddings
    Xtr, ytr, _      = build_xy(by_species, train_pairs)
    Xte, yte, te_ids = build_xy(by_species, test_pairs)

    # 3. Train SVM
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm", SVC(kernel="rbf", class_weight="balanced", probability=True))
    ])
    clf.fit(Xtr, ytr)

    # 4. Predict
    yhat  = clf.predict(Xte)
    probs = clf.predict_proba(Xte)
    conf  = probs.max(axis=1)

    # 5. Per-specimen table
    results = pd.DataFrame({
        "sample_id": te_ids,
        "true_label": yte,
        "pred_label": yhat,
        "pred_confidence": conf
    })
    results["correct"] = (results.true_label == results.pred_label).astype(int)

    # 6. Per-species table
    per_species = (results.assign(one=1)
                   .groupby("true_label")
                   .agg(n_test=("one","sum"),
                        n_correct=("correct","sum"))
                   .assign(accuracy_species=lambda d: d.n_correct / d.n_test)
                   .reset_index()
                   .rename(columns={"true_label":"species"}))

    # 7. Metrics
    overall_acc = float(results["correct"].mean())
    macro_acc   = float(per_species["accuracy_species"].mean())

    # 8. Save per-seed artifacts
    seed_dir = RESULTS_ROOT / f"seed_{seed:03d}"
    seed_dir.mkdir(parents=True, exist_ok=True)

    results.to_csv(seed_dir / "predictions.csv", index=False)
    per_species.to_csv(seed_dir / "per_species.csv", index=False)

    try:
        import bioclip as _bioclip_mod
        bioclip_version = getattr(_bioclip_mod, "__version__", "unknown")
    except Exception:
        bioclip_version = "unknown"

    summary = {
        "overall_accuracy": overall_acc,
        "macro_accuracy": macro_acc,
        "n_test": int(len(results)),
        "K": int(K),
        "MIN_TOTAL": int(MIN_TOTAL),
        "seed": int(seed),
        "device": str(DEVICE),
        "bioclip_version": bioclip_version,
        "timestamp": int(time.time())
    }
    with open(seed_dir / "summary.json", "w") as f:
        json.dump(summary, f, indent=2)

    with open(seed_dir / "split.json", "w") as f:
        json.dump({
            "train_pairs": train_pairs,
            "test_pairs":  test_pairs
        }, f, indent=2)

    return {
        "seed": seed,
        "overall_accuracy": overall_acc,
        "macro_accuracy": macro_acc,
        "n_test": int(len(results))
    }

# ---- loop over seeds ----
all_rows = []
for s in tqdm(range(100), desc="Monte Carlo 5-shot SVM runs"):
    row = run_one_seed(s)
    all_rows.append(row)

# ---- aggregate ----
agg = pd.DataFrame(all_rows).sort_values("seed").reset_index(drop=True)
agg.to_csv(RESULTS_ROOT / "metrics_5shot_seeds.csv", index=False)

run_meta = {
    "seeds": list(range(100)),
    "K": int(K),
    "MIN_TOTAL": int(MIN_TOTAL),
    "n_species": int(len(included_species)),
    "emb_cache_dir": str(EMB_CACHE.resolve()),
    "results_root": str(RESULTS_ROOT.resolve()),
    "device": str(DEVICE)
}
with open(RESULTS_ROOT / "run_meta.json", "w") as f:
    json.dump(run_meta, f, indent=2)

# ---- print aggregate stats ----
with pd.option_context('display.float_format', '{:.4f}'.format):
    print("\n=== Aggregate over 100 seeds ===")
    print(agg[["overall_accuracy","macro_accuracy"]].describe())

mu = agg["macro_accuracy"].mean()
sd = agg["macro_accuracy"].std(ddof=1)
n  = agg["macro_accuracy"].notna().sum()
ci95 = 1.96 * sd / np.sqrt(max(n,1))
print(f"\nMacro accuracy mean ± 95% CI: {mu:.4f} ± {ci95:.4f}  (n={n})")

Monte Carlo 5-shot SVM runs: 100%|██████████| 100/100 [00:07<00:00, 13.52it/s]


=== Aggregate over 100 seeds ===
       overall_accuracy  macro_accuracy
count          100.0000        100.0000
mean             0.8336          0.7778
std              0.0308          0.0897
min              0.7016          0.5275
25%              0.8156          0.7575
50%              0.8336          0.8117
75%              0.8517          0.8374
max              0.9042          0.8852

Macro accuracy mean ± 95% CI: 0.7778 ± 0.0176  (n=100)
